# 2024 COMP90042 Project
*Make sure you change the file name with your group id.*

# Readme
*If there is something to be noted for the marker, please mention here.*

*If you are planning to implement a program with Object Oriented Programming style, please put those the bottom of this ipynb file*

# 1.DataSet Processing
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [45]:
LOCAL_DEV = True # to switch between developing locally and on colab

if not LOCAL_DEV:
    # TODO: need to upload data files on Google Drive?
    from google.colab import drive
    drive.mount('/content/drive')

In [46]:
pip install contractions

Note: you may need to restart the kernel to use updated packages.


In [47]:
#Imports
import numpy as np
import torch
import pandas as pd

In [48]:
#visualising training data
if LOCAL_DEV:
    train = pd.read_json("../data/train-claims.json") # for local dev
    
else:
    train = pd.read_json("/content/drive/MyDrive/data/train-claims.json") # on colab
train = train.transpose()
train.head()


,claim_text,claim_label,evidences
claim-1937,Not only is there no scientific evidence that ...,DISPUTED,"[evidence-442946, evidence-1194317, evidence-1..."
claim-126,El Niño drove record highs in global temperatu...,REFUTES,"[evidence-338219, evidence-1127398]"
claim-2510,"In 1946, PDO switched to a cool phase.",SUPPORTS,"[evidence-530063, evidence-984887]"
claim-2021,Weather Channel co-founder John Coleman provid...,DISPUTED,"[evidence-1177431, evidence-782448, evidence-5..."
claim-2449,"""January 2008 capped a 12 month period of glob...",NOT_ENOUGH_INFO,"[evidence-1010750, evidence-91661, evidence-72..."


In [49]:
if LOCAL_DEV:
    test = pd.read_json("../data/test-claims-unlabelled.json") # for local dev
    
else:
    test = pd.read_json("/content/drive/MyDrive/data/test-claims-unlabelled.json") # on colab
test = test.transpose()
test.head()

,claim_text
claim-2967,The contribution of waste heat to the global c...
claim-979,“Warm weather worsened the most recent five-ye...
claim-1609,Greenland has only lost a tiny fraction of its...
claim-1020,“The global reef crisis does not necessarily m...
claim-2599,Small amounts of very active substances can ca...


In [50]:
#visualising evidence data
if LOCAL_DEV:
    evidence = pd.read_json("../data/evidence.json",typ='series')
else:
    evidence = pd.read_json("/content/drive/MyDrive/data/evidence.json",typ='series')

In [51]:
print(len(evidence))
evidence.head()

1208827


evidence-0    John Bennet Lawes, English entrepreneur and ag...
evidence-1    Lindberg began his professional career at the ...
evidence-2    ``Boston (Ladies of Cambridge)'' by Vampire We...
evidence-3    Gerald Francis Goyer (born October 20, 1936) w...
evidence-4    He detected abnormalities of oxytocinergic fun...
dtype: object

In [52]:
import string
import contractions
from collections import defaultdict
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asura\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asura\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [53]:
def preprocess_data(data: pd.Series) -> pd.Series:
  preprocessed_data = {}
  stop_words = set(stopwords.words('english'))
  stop_words.remove('not')
  for id, text in data.items():
    text = text.lower()
    text = contractions.fix(text)
    tokens = word_tokenize(text)
    wnl = WordNetLemmatizer()
    lemmatized_tokens = [wnl.lemmatize(word) for word in tokens if word.isalpha() and word not in stop_words]
    preprocessed_data[id] = " ".join(lemmatized_tokens)

  return pd.Series(preprocessed_data)

processed_evidence = preprocess_data(evidence)

test_claims = test['claim_text']
processed_test = preprocess_data(test_claims)
processed_test.head()

claim-2967               contribution waste heat global climate
claim-979     warm weather worsened recent drought included ...
claim-1609                greenland lost tiny fraction ice mass
claim-1020    global reef crisis not necessarily mean extinc...
claim-2599     small amount active substance cause large effect
dtype: object

In [54]:
processed_evidence = processed_evidence[processed_evidence.str.strip().str.len() > 0]

In [55]:
SPECIAL_TOKEN = ' <SPE_TOKEN> '
def prepareTrainData(n):
    train_claims = preprocess_data(train['claim_text'])
    processed_train_claim = preprocess_data(train_claims)
    text_lst = []
    label_lst = []
    for i in range(len(train)):
        train_claim = processed_train_claim[i]
        evidences = train.iloc[i]['evidences']
        for j in evidences:
            if j in processed_evidence.index :
                text = train_claim + SPECIAL_TOKEN + processed_evidence[j]
                text_lst.append(text)
                label_lst.append('related')
        filtered_evi = processed_evidence[~processed_evidence.index.isin(evidences)]
        random_evidence = filtered_evi.sample(n)
        for k in random_evidence:
            text = train_claim + SPECIAL_TOKEN + k
            text_lst.append(text)
            label_lst.append('unrelated')
    claim_evi_label = {'text': text_lst, 'label': label_lst}
    return pd.DataFrame(claim_evi_label)

train_claims = train['claim_text']
processed_train_claim = preprocess_data(train_claims)
preparedTrain = prepareTrainData(10)
preparedTrain.head()

C:\Users\Asura\AppData\Local\Temp\ipykernel_26768\2039351849.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  train_claim = processed_train_claim[i]


,text,label
0,not scientific evidence pollutant higher conce...,related
1,not scientific evidence pollutant higher conce...,related
2,not scientific evidence pollutant higher conce...,related
3,not scientific evidence pollutant higher conce...,unrelated
4,not scientific evidence pollutant higher conce...,unrelated


In [56]:
preparedTrain.head(10)

,text,label
0,not scientific evidence pollutant higher conce...,related
1,not scientific evidence pollutant higher conce...,related
2,not scientific evidence pollutant higher conce...,related
3,not scientific evidence pollutant higher conce...,unrelated
4,not scientific evidence pollutant higher conce...,unrelated
5,not scientific evidence pollutant higher conce...,unrelated
6,not scientific evidence pollutant higher conce...,unrelated
7,not scientific evidence pollutant higher conce...,unrelated
8,not scientific evidence pollutant higher conce...,unrelated
9,not scientific evidence pollutant higher conce...,unrelated


In [57]:
# need later versions for torchtext.transforms and special
#pip install torchtext==0.18.0 

# Two steps for the this task
# first. find all relavent evidence, either use contextual embedding or similarity scoring
# second. classify the evidents into 4 classes.

# 2. Model Implementation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [58]:
pip install -U scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import torchtext
from torchtext.data.utils import get_tokenizer
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.transforms
from torch.nn.utils.rnn import pad_sequence
# No module named 'torchtext.transforms' ?

g:\Ana\envs\nlp\Lib\site-packages\torchtext\data\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
g:\Ana\envs\nlp\Lib\site-packages\torchtext\vocab\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
g:\Ana\envs\nlp\Lib\site-packages\torchtext\utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last rele

In [60]:
# tokenize and vectorise training data for relevance classification

tokenizer = get_tokenizer("basic_english")
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(preparedTrain['text']), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

text_pipeline = lambda x: vocab(tokenizer(x))
label_transform = torchtext.transforms.LabelToIndex({'related': 0, 'unrelated': 1})

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    text_list, label_list, offsets = [], [], [0]
    for _text, _label in batch:
        label_list.append(label_transform(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    # should stack instead of concat?
    #text_list = torch.cat(text_list)
   
    # pad sequences to make them the same length 
    padded_sequences = pad_sequence(text_list, batch_first=True)

    return padded_sequences.to(device), label_list.to(device), offsets.to(device)

class TextDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        text = row['text'] 
        label = row['label'] 
        return text, label

full_dataset = TextDataset(preparedTrain)
# train test split
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=20, shuffle=False, collate_fn=collate_batch)
val_dataloader = DataLoader(val_dataset, batch_size=20, shuffle=False, collate_fn=collate_batch)

In [61]:
# initialise the relevance classification model
# NOTE: run the OOP code first!
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ntokens = len(vocab) # TODO: verify correctness of this

emsize = 200 # embedding dimension
nhid = 200 # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2 # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2 # the number of heads in the multiheadattention models
dropout = 0.2 # the dropout value

model = TransformerClassificationModel(ntokens, emsize, nhead, nhid, nlayers, 2, dropout).to(device)

g:\Ana\envs\nlp\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [62]:
# train model
import math
import torch.nn as nn
criterion = nn.CrossEntropyLoss()
lr = 1.0 # learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

# TODO: figure out what this does and decide if it's needed in train
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

import time

def train_model(train_data_loader, model):

    model.train() # Turn on the train mode
    for inputs, labels, offsets in train_data_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        clipping_value = 1 # arbitrary value of your choosing
        # training unstable? https://stackoverflow.com/questions/66625645/why-does-my-pytorch-nn-return-a-tensor-of-nan
        torch.nn.utils.clip_grad_norm_(model.parameters(), clipping_value) # https://stackoverflow.com/questions/54716377/how-to-do-gradient-clipping-in-pytorch
        optimizer.step()


def evaluate(val_data_loader, eval_model):
    eval_model.eval() # Turn on the evaluation mode
    total_loss = 0.

    with torch.no_grad():
       
        for inputs, labels, offsets in val_data_loader:
          
            output = eval_model(inputs)
            # output_flat = output.view(-1, ntokens) do we need to do this?
            total_loss += len(inputs) * criterion(output, labels).item()
    return total_loss / (len(val_data_loader) - 1)




best_val_loss = float("inf")
epochs = 3 # The number of epochs
best_model = None
#train(dataloader, model)
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()
    # TODO: PUT DATA HERE
    train_model(train_dataloader, model)
    val_loss = evaluate(val_dataloader, model) 
    print('-' * 89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
          'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                     val_loss, math.exp(val_loss)))
    print('-' * 89)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model

    scheduler.step()


g:\Ana\envs\nlp\Lib\site-packages\torch\nn\functional.py:5504: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


-----------------------------------------------------------------------------------------
| end of epoch   1 | time:  7.45s | valid loss 12.95 | valid ppl 418902.31
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   2 | time:  6.05s | valid loss  9.09 | valid ppl  8906.35
-----------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------
| end of epoch   3 | time:  5.80s | valid loss  7.88 | valid ppl  2647.04
-----------------------------------------------------------------------------------------


In [63]:
# sanity check

print(val_dataset[0])
val_batch = next(iter(val_dataloader))
print(val_batch[0][0]) # first input
print(val_batch[1]) # labels

test_outputs = model(val_batch[0])
predicted_labels = torch.argmax(test_outputs, dim=1)

print(predicted_labels)
    

('forget global warming activist would lead not even close hottest year record <SPE_TOKEN> global warming rise average temperature earth climate system', 'related')
tensor([3328,    4,    2, 1918,   30,  242,    7,   94,  469,  710,    6,   20,
           1,    4,    2,   19,   65,    5,   23,    3,   91,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0], device='cuda:0')
tensor([0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1],
       device='cuda:0')
tensor([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1],
       device='cuda:0')


# 3.Testing and Evaluation
(You can add as many code blocks and text blocks as you need. However, YOU SHOULD NOT MODIFY the section title)

In [64]:
def accuracy(val_data_loader, eval_model):
    model.eval()
    total_correct = 0
    total_samples = 0

    for inputs, labels, offsets in val_data_loader:
        outputs = model(inputs)
        predicted_labels = torch.argmax(outputs, dim=1)
        total_samples += labels.size(0)
        total_correct += (predicted_labels == labels).sum().item()

    accuracy = total_correct / total_samples
    return accuracy

test_loss = evaluate(val_dataloader, best_model) 
test_acc = accuracy(val_dataloader, best_model)

print('=' * 89)
print('| End of training | test loss {:5.2f} | test ppl {:8.2f} | test accuracy: {:5.2f}'.format(
    test_loss, math.exp(test_loss), test_acc))
print('=' * 89)




| End of training | test loss  7.88 | test ppl  2647.04 | test accuracy:  0.84


In [65]:
# Try to use the model on unseen test claims:
# Get all evidences that are likely to be relevant using a similarity score

# Vectorizing preprocessed text
# TODO: replace Tfidf with contextual embedding
vectorizer = TfidfVectorizer()
all_texts = pd.concat([processed_evidence, processed_train_claim])
vectorizer.fit(all_texts)

evidence_tfidf = vectorizer.transform(processed_evidence)
test_tfidf = vectorizer.transform(processed_test)

similarity_matrix = cosine_similarity(test_tfidf, evidence_tfidf)

def getTopN(similarity_matrix, test, evidence, n):
  test = test.to_frame(name='claim_text')
  top_indices = np.argsort(-similarity_matrix, axis = 1)[:, :n]
  top_evidence = [[str(evidence.index[i]) for i in row] for row in top_indices]
  test['evidences'] = top_evidence
  return test

test_with_evi = getTopN(similarity_matrix, processed_test, processed_evidence, 10)
test_with_evi.head()

,claim_text,evidences
claim-2967,contribution waste heat global climate,"[evidence-308923, evidence-213569, evidence-63..."
claim-979,warm weather worsened recent drought included ...,"[evidence-178433, evidence-421870, evidence-43..."
claim-1609,greenland lost tiny fraction ice mass,"[evidence-962481, evidence-1200633, evidence-7..."
claim-1020,global reef crisis not necessarily mean extinc...,"[evidence-642301, evidence-161852, evidence-67..."
claim-2599,small amount active substance cause large effect,"[evidence-834109, evidence-1175545, evidence-8..."


In [66]:

# format test data to be put into model
def prepareTestData():
    tfidf_claim = test_with_evi['claim_text']
    tfidf_evi = test_with_evi['evidences']
    text_lst = []
    for i in range(len(tfidf_claim)):
        test_claim = tfidf_claim[i]
        evidences = tfidf_evi[i]
        for j in evidences:
            text = test_claim + SPECIAL_TOKEN + processed_evidence[j]
            text_lst.append(text)
    claim_evi = {'text': text_lst, 'label':'unrelated'} # still include the label field to avoid index error
    return pd.DataFrame(claim_evi)

preparedTest = prepareTestData()
preparedTest.head()
test_dataset = TextDataset(preparedTest)
test_dataloader = DataLoader(test_dataset, batch_size=20, shuffle=False, collate_fn=collate_batch)

C:\Users\Asura\AppData\Local\Temp\ipykernel_26768\418587428.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  test_claim = tfidf_claim[i]
C:\Users\Asura\AppData\Local\Temp\ipykernel_26768\418587428.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  evidences = tfidf_evi[i]


In [67]:
# manually check the results since we don't have the labels:
from collections import Counter
print(test_dataset[0])
test_batch = next(iter(test_dataloader))
print(test_batch[0][0]) # first input

test_outputs = best_model(test_batch[0])
predicted_labels = torch.argmax(test_outputs, dim=1)
print(predicted_labels)


def get_all_predictions(model, test_loader):
    model.eval()
    all_predictions = []
    for inputs, _, _ in test_loader:
        outputs = model(inputs)
        predicted_labels = torch.argmax(outputs, dim=1)
        all_predictions.extend(predicted_labels.cpu().numpy())
    return all_predictions

# check class imbalance
def count_labels(predictions):
    label_counts = Counter(predictions)
    return label_counts


# get all predictions

all_predictions = get_all_predictions(best_model, test_dataloader)
print(count_labels(all_predictions))



('contribution waste heat global climate <SPE_TOKEN> global forcing waste heat', 'unrelated')
tensor([1028, 1575,   50,    4,    3,    1,    4,  502, 1575,   50,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0], device='cuda:0')
tensor([1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       device='cuda:0')
Counter({1: 1028, 0: 502})


In [68]:
# update test_with_evi with model results

def filter_relevant_evidences(claim_evidences, model_classifications, default=True):
    claims = claim_evidences['claim_text']
    evidences = claim_evidences['evidences']
    filtered_evidences = []

    classifications_index = 0
    for i in range(len(claims)):
        claim = claims[i]
        curr_evidences = evidences[i]
        curr_filtered = []
        for evidence in curr_evidences:
            # 0 => related
            if model_classifications[classifications_index] == 0:
                curr_filtered.append(evidence)

            classifications_index += 1

        if default and len(curr_filtered) == 0:
            # keep the most similar evidence if all are deemed unrelated
            curr_filtered.append(curr_evidences[0]) 

        filtered_evidences.append(curr_filtered)

    claim_evi = {'text': claims, 'label': filtered_evidences} # still include the label field to avoid index error
    return pd.DataFrame(claim_evi)

filtered_claim_evidences = filter_relevant_evidences(test_with_evi, all_predictions)
filtered_claim_evidences.head()

C:\Users\Asura\AppData\Local\Temp\ipykernel_26768\494974565.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  claim = claims[i]
C:\Users\Asura\AppData\Local\Temp\ipykernel_26768\494974565.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  curr_evidences = evidences[i]


,text,label
claim-2967,contribution waste heat global climate,"[evidence-213569, evidence-631339, evidence-19..."
claim-979,warm weather worsened recent drought included ...,"[evidence-178433, evidence-421870, evidence-43..."
claim-1609,greenland lost tiny fraction ice mass,"[evidence-1200633, evidence-726093, evidence-6..."
claim-1020,global reef crisis not necessarily mean extinc...,"[evidence-670186, evidence-542625, evidence-16..."
claim-2599,small amount active substance cause large effect,"[evidence-860747, evidence-1045673]"


In [69]:
print(filtered_claim_evidences["label"])

claim-2967    [evidence-213569, evidence-631339, evidence-19...
claim-979     [evidence-178433, evidence-421870, evidence-43...
claim-1609    [evidence-1200633, evidence-726093, evidence-6...
claim-1020    [evidence-670186, evidence-542625, evidence-16...
claim-2599                  [evidence-860747, evidence-1045673]
                                    ...                        
claim-293                                    [evidence-1112386]
claim-910                                     [evidence-197221]
claim-2815    [evidence-786294, evidence-700183, evidence-12...
claim-1652    [evidence-386828, evidence-442810, evidence-35...
claim-1212                                    [evidence-162296]
Name: label, Length: 153, dtype: object


In [71]:
# manually check some of them to ensure they are somewhat correct

# need to keep at least one for this one
print(test_claims['claim-2967'])
print('=' * 89)

print(test_claims['claim-979'])
print(evidence['evidence-178433'])
print(evidence['evidence-421870'])
print('=' * 89)

print(test_claims['claim-1609'])
print(evidence['evidence-382341'])
print(evidence['evidence-726093'])

print('=' * 89)

print(test_claims['claim-1020'])
print(evidence['evidence-382341'])
print(evidence['evidence-542625'])
print('=' * 89)
print(test_claims['claim-2599'])
print(evidence['evidence-860747'])

The contribution of waste heat to the global climate is 0.028 W/m2.
“Warm weather worsened the most recent five-year drought, which included the driest four-year period on record in terms of statewide precipitation.
Another form of severe weather is drought, which is a prolonged period of persistently dry weather (that is, absence of precipitation).
This is worsened by extreme weather events caused by climate change.
Greenland has only lost a tiny fraction of its ice mass.
With widespread degradation of highly biodiverse habitats such as coral reefs and rainforests, as well as other areas, the vast majority of these extinctions are thought to be "undocumented", as the species are undiscovered at the time of their extinction, or no one has yet discovered their extinction.
The Greenland ice sheet (, ) is a vast body of ice covering , roughly 80% of the surface of Greenland.
“The global reef crisis does not necessarily mean extinction for coral species.
With widespread degradation of high

In [8]:
import re
import pandas as pd
from collections import defaultdict
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
wnl = WordNetLemmatizer()
evidence = pd.read_json("../data/evidence.json",typ='series')
evidence_pd = pd.DataFrame(evidence).reset_index()
evidence_pd.columns = ['claim_id', 'claims']
evidence_pd["pro_evidence"] = evidence_pd["claims"].apply(str.lower)
evidence_pd["pro_evidence"] = evidence_pd["pro_evidence"].apply(lambda x: re.sub(r'[^\w\s]', '', x))
evidence_pd["pro_evidence"] = evidence_pd["pro_evidence"].apply(lambda x: ' '.join(wnl.lemmatize(word) for word in x.split( ) if word not in stopwords and word.isalpha()))

train_data = pd.read_json("../data/train-claims.json")
train_data = train_data.transpose()
train_data["pro_claim_text"] = train_data['claim_text'].apply(str.lower)
train_data["pro_claim_text"] = train_data["pro_claim_text"].apply(lambda x: re.sub(r'[^\w\s]', '', x))
train_data["pro_claim_text"] = train_data["pro_claim_text"].apply(lambda x: ' '.join(wnl.lemmatize(word) for word in x.split( ) if word not in stopwords and word.isalpha()))

train_data["pro_evidences"] = train_data["evidences"].apply(lambda x:  [evidence_pd.iloc[int(re.sub("[^0-9]", "", i))]["pro_evidence"] for i in x])
train_data["evidence_ids"] = train_data["evidences"].apply(lambda x:  [int(re.sub("[^0-9]", "", i)) for i in x])
train_data.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Asura\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Asura\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Asura\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,claim_text,claim_label,evidences,pro_claim_text,pro_evidences,evidence_ids
claim-1937,Not only is there no scientific evidence that ...,DISPUTED,"[evidence-442946, evidence-1194317, evidence-1...",scientific evidence pollutant higher concentra...,[high concentration time atmospheric concentra...,"[442946, 1194317, 12171]"
claim-126,El Niño drove record highs in global temperatu...,REFUTES,"[evidence-338219, evidence-1127398]",el niño drove record high global temperature s...,[climate change due natural force human activi...,"[338219, 1127398]"
claim-2510,"In 1946, PDO switched to a cool phase.",SUPPORTS,"[evidence-530063, evidence-984887]",pdo switched cool phase,[evidence reversal prevailing polarity meaning...,"[530063, 984887]"
claim-2021,Weather Channel co-founder John Coleman provid...,DISPUTED,"[evidence-1177431, evidence-782448, evidence-5...",weather channel cofounder john coleman provide...,[convincing scientific evidence human release ...,"[1177431, 782448, 540069, 352655, 1007867]"
claim-2449,"""January 2008 capped a 12 month period of glob...",NOT_ENOUGH_INFO,"[evidence-1010750, evidence-91661, evidence-72...",january capped month period global temperature...,"[average temperature c f, iranian persian cale...","[1010750, 91661, 722725, 554161, 430839]"


In [28]:

id2label = {0:"SUPPORTS", 1:"NOT_ENOUGH_INFO", 2:"REFUTES", 3:"DISPUTED"}
label2id = {"SUPPORTS":0, "NOT_ENOUGH_INFO":1, "REFUTES":2, "DISPUTED":3}
id2token = {0:"<pad>", 1:"<cls>", 2:"<sep>", 3:"<unk>"}
token2id = {"<pad>": 0, "<cls>": 1, "<sep>": 2, "<unk>": 3}

In [29]:
from collections import Counter
train_words = (train_data['pro_claim_text'].apply(lambda x : x.split())).tolist()
evidence_words = (evidence_pd["pro_evidence"].apply(lambda x : x.split())).tolist()
train_words1 = [x for xs in train_words for x in xs ]
evidence_words1 = [x for xs in evidence_words for x in xs ]
vocab = list(set(train_words1 + evidence_words1))
word_counts = Counter(train_words1 + evidence_words1)
idx = 4
for i, j in word_counts.items():
  if j > 5:
    id2token[idx] = i
    token2id[i] = idx
    idx +=1

In [30]:
idx = 4
for i, j in word_counts.items():
  if j > 5:
    id2token[idx] = i
    token2id[i] = idx
    idx +=1

In [31]:
def wordtointvec(inputs, token2id):
  lst = []
  for i in inputs:

    sent = []
    for w in i.split(" "):
      sent.append(token2id.get(w, token2id["<unk>"]))
    lst.append(sent)

  return lst
def wordtointvec4evi(inputs, token2id):
  lst = []
  for x in inputs:
    sent = []
    for i in x:
      insent = []
      for w in i.split(" "):
        insent.append(token2id.get(w, token2id["<unk>"]))
      sent.append(insent)

    lst.append(sent)

  return lst


In [32]:
def addpadding(l, inputs, token2id):
  lst = []
  for i in inputs:
    if len(i) < l:
      lst.append([token2id["<cls>"]]+ i + [token2id["<sep>"]] + (l-len(i)) * [token2id["<pad>"]])
    else:

      lst.append([token2id["<cls>"]]+ i[:l] + [token2id["<sep>"]] )
  return lst


In [33]:
train_vec = wordtointvec(train_data['pro_claim_text'],token2id)
train_evi_vec = wordtointvec4evi(train_data['pro_evidences'],token2id)
#evi_vec = wordtointvec(evidence_pd["pro_evidence"],token2id)
#dev_vec = wordtointvec(dev['pro_claim_text'],token2id)

In [34]:
train_in = addpadding(34, train_vec, token2id)
#train_evi_in = addpadding(100, train_evi_vec, token2id)
#evi_in = addpadding(100, evi_vec, token2id)
#dev_in =addpadding(34, dev_vec, token2id)
max_len = 280
for i in range(len(train_in)):
  for j in train_evi_vec[i]:
    train_in[i].append(token2id["<sep>"])
    train_in[i].extend(j)
  train_in[i].append(token2id["<sep>"])
  if len(train_in[i]) < max_len:

    train_in[i].extend([token2id["<pad>"]] * (max_len - len(train_in[i])))

print(train_in[0])


[1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 20, 8, 52, 110, 8, 851, 278, 279, 1843, 14, 15, 2421, 8, 765, 7, 1001, 3226, 2957, 19214, 3, 8271, 38097, 154, 2, 13, 237, 95, 299, 1791, 8, 765, 4353, 103, 22355, 170, 784, 10925, 122, 121, 7018, 20717, 2, 7, 278, 279, 8, 22501, 894, 13, 2116, 6789, 371, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [35]:
# max_len = 550
# train_in = [] 
# for i in range(len(processed_train_claim)):
#   train_in.append(processed_train_claim[i])

# for i in range(len(processed_train_claim)):
#   for j in train_data["pro_evidences"][i]:
#     train_in[i] += "<sep>"
#     train_in[i] += j
#   train_in[i]+= "<sep>"
#   if len(train_in[i]) < max_len:
#     for j in range(max_len - len(train_in[i])):
#       train_in[i] += " "
#       train_in[i] += "<pad>"
# train_in = vectorizer.transform(train_in).toarray()
# print(train_in[0])

In [36]:
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn

from torch.utils.data import DataLoader, Dataset
class TrainDataset(Dataset):
  def __init__(self, text_data, label_data):
    self.text_data = text_data
    self.label_data = [label2id[i] for i in label_data]
    

  def __getitem__ (self, index):
    return [self.text_data[index], self.label_data[index]]
  def __len__(self):
    return len(self.text_data)

  def collate_fn(self, data):
    q = []
    #evi = []
    labels = []
    for sen, lab in data:
      q.append(sen)
      labels.append(lab)
    batch_encoding = {}
    batch_encoding["queries"] =  torch.LongTensor(q)

    batch_encoding["labels"] = torch.LongTensor(labels)
    return batch_encoding

In [37]:
train_dataset = TrainDataset(train_in, train_data["claim_label"])
dataloader = DataLoader(train_dataset, batch_size = 20, shuffle = True, num_workers = 0,collate_fn=train_dataset.collate_fn)

In [38]:
class AttentionBlock(nn.Module):
  def __init__(self, embed_dim, n_heads, drop_prob):
    super(AttentionBlock , self).__init__()
    self.att = nn.MultiheadAttention(embed_dim, n_heads)
    self.ffn = nn.Sequential(
        nn.Linear(embed_dim, 4*embed_dim),
        nn.ReLU(),
        nn.Linear(4*embed_dim, embed_dim)
    )
    self.ln1 = nn.LayerNorm(embed_dim, eps = 1e-6)
    self.ln2 = nn.LayerNorm(embed_dim, eps = 1e-6)
    self.dropout1 = nn.Dropout(drop_prob)
    self.dropout2 = nn.Dropout(drop_prob)
  def forward(self, inputs):
    attn_out, _ = self.att(inputs, inputs,inputs)
    attn_out = self.dropout1(attn_out)
    out1 = self.ln1(inputs + attn_out)
    ffn_output = self.ffn(out1)
    ffn_output = self.dropout2(ffn_output)
    return self.ln2(out1 + ffn_output)
class ClassificationModel(nn.Module):
  def __init__(self, embed_dim, wordlist_size, block_size, drop_prob, n_classes, n_heads):
    super(ClassificationModel , self).__init__()
    self.token_embed = nn.Embedding(wordlist_size, embed_dim)
    self.pos_embed = nn.Embedding(block_size, embed_dim)
    self.AttentionBlock = AttentionBlock(embed_dim, n_heads, drop_prob)
    self.ln1 = nn.LayerNorm(embed_dim)
    self.ln2 = nn.LayerNorm(embed_dim)
    self.classifier = nn.Sequential(
        nn.Linear(embed_dim, embed_dim),
        nn.ReLU(),
        nn.Dropout(drop_prob),
        nn.ReLU(),
        nn.Linear(embed_dim, n_classes),
        nn.Softmax(dim = 1)
    )
  def forward(self, token):
    r,c = token.shape
    emb = self.token_embed(token)
    emb = emb+ self.pos_embed(torch.arange(c, device = device))
    out = self.AttentionBlock(emb)
    out = out.mean(dim = 1)
    out = self.classifier(out)
    return out

In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
classification_model = ClassificationModel(embed_dim = 512, wordlist_size = len(id2token), block_size = 280, drop_prob = 0.1, n_classes = 4, n_heads = 4)
classification_model.to(device)
lr = 0.0001
optimizer = torch.optim.Adam(classification_model.parameters(), lr=lr)
loss_t = nn.CrossEntropyLoss()

In [40]:

def train():
  classification_model.train()
  total_loss = 0.
  #start_time = time.time()
  for (i, targets) in enumerate(dataloader):
    optimizer.zero_grad()
    out = classification_model(targets["queries"].cuda())
    loss = loss_t(out, targets["labels"].cuda())
    #loss = loss
    #torch.nn.utils.clip_grad_norm_()

    loss.backward()
    optimizer.step()
    total_loss += loss.item()


def evaluate(model, input, out):
    lst = []
    start = 0
    batch_size = 50
    in_len = len(input[0])
    model.eval()
    correct_count = 0
    while start < len(out):
        end = min(start+ batch_size, len(out))

        model_input = torch.LongTensor(input[start:end]).view(-1, in_len).cuda()

        model_output = model(model_input)
        model_output = torch.argmax(model_output, 1).tolist()

        for i, j in zip(model_output, out[start: end]):
            if i == j:
                correct_count += 1

        start = end
    lst = correct_count / len(out)
    print("\n")
    print("Classification Accuracy: %.3f" % lst)
    print("\n")

    model.train()
    return lst


In [43]:
def predict( input, model):
    lst = []
    start = 0
    batch_size = 50
    in_len = len(input[0])
    model.eval()

    
    
    while start < len(input):
        end = min(start+ batch_size, len(input))
        
        model_input= torch.LongTensor(input[start:end]).view(-1, in_len).cuda()
        
        model_output = model(model_input)
        model_output = torch.argmax(model_output, 1).tolist()
        lst.extend(model_output)
        
        start = end

    return lst

In [41]:

epochs = 3
for epoch in range(epochs):
  loss = 0
  train()
  f_score = evaluate(classification_model, dev_in, dev_label)
  print("fscore", f_score)

NameError: name 'dev_in' is not defined

In [75]:
filtered_claim_evidences["pro_evidences"] = filtered_claim_evidences["label"].apply(lambda x:  [evidence_pd.iloc[int(re.sub("[^0-9]", "", i))]["pro_evidence"] for i in x])

In [76]:
filtered_claim_evidences

,text,label,pro_evidences
claim-2967,contribution waste heat global climate,"[evidence-213569, evidence-631339, evidence-19...",[thus waste heat engine may one least expensiv...
claim-979,warm weather worsened recent drought included ...,"[evidence-178433, evidence-421870, evidence-43...",[another form severe weather drought prolonged...
claim-1609,greenland lost tiny fraction ice mass,"[evidence-1200633, evidence-726093, evidence-6...",[land ice sheet antarctica greenland losing ma...
claim-1020,global reef crisis not necessarily mean extinc...,"[evidence-670186, evidence-542625, evidence-16...",[bleaching coral great barrier reef killed per...
claim-2599,small amount active substance cause large effect,"[evidence-860747, evidence-1045673]",[loosely divided cause effect mitigation notin...
...,...,...,...
claim-293,measuring equipment get old need replacing oft...,[evidence-1112386],[opened replacing old georgvonneumayerstation]
claim-910,cement iron steel petroleum refining industry ...,[evidence-197221],[specifically reduction iron steel production ...
claim-2815,new study surface warming solar cycle found ti...,"[evidence-786294, evidence-700183, evidence-12...",[solar cycle solar cycle since extensive recor...
claim-1652,strong effect observed many different measurement,"[evidence-386828, evidence-442810, evidence-35...",[major aspect climate change demonstrated dire...


In [77]:
filtered_claim_evidences
test_vec = wordtointvec(filtered_claim_evidences["text"],token2id)
test_in = addpadding(34, test_vec, token2id)
test_evi_vec = wordtointvec4evi(filtered_claim_evidences['pro_evidences'],token2id)
max_len = 280
for i in range(len(test_in)):
  for j in test_evi_vec[i]:
    test_in[i].append(token2id["<sep>"])
    test_in[i].extend(j)
  test_in[i].append(token2id["<sep>"])
  if len(test_in[i]) < max_len:

    test_in[i].extend([token2id["<pad>"]] * (max_len - len(test_in[i])))

print(test_in[0])

[1, 2800, 1935, 713, 21, 121, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1828, 1935, 713, 3294, 25, 888, 1250, 2312, 2473, 4731, 1935, 713, 1079, 150, 2, 4253, 4708, 3, 4387, 1768, 904, 78752, 713, 593, 1935, 713, 149, 713, 2, 126, 51, 455, 403, 2816, 1615, 6365, 1935, 713, 381, 1017, 22580, 1738, 1206, 573, 713, 1935, 186, 2, 610, 455, 158, 3789, 11059, 40, 121, 122, 227, 48, 1935, 713, 5989, 2, 4642, 1935, 713, 3294, 46774, 715, 4239, 3294, 2536, 902, 885, 4239, 2878, 1935, 713, 2, 339, 78752, 1935, 713, 1206, 371, 843, 729, 2, 121, 122, 898, 21, 551, 121, 52, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [78]:
test_classes = predict(test_in, classification_model)

In [ ]:
test_classes

## Object Oriented Programming codes here

*You can use multiple code snippets. Just add more if needed*

In [44]:
import math
import torch
import torch.nn as nn
import torch.nn.functional as F

# model adpated from workshop 8
class TransformerClassificationModel(nn.Module):
    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, num_classes, dropout=0.5):
        super(TransformerClassificationModel, self).__init__()
        from torch.nn import TransformerEncoder, TransformerEncoderLayer
        self.model_type = 'Transformer'
        self.src_mask = None
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        encoder_layers = TransformerEncoderLayer(ninp, nhead, nhid, dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.encoder = nn.Embedding(ntoken, ninp)
        self.ninp = ninp
        self.classification_head = nn.Linear(ninp, num_classes)  # added classification head
        self.init_weights()

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.classification_head.bias.data.zero_()
        self.classification_head.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        if self.src_mask is None or self.src_mask.size(0) != len(src):
            device = src.device
            mask = self._generate_square_subsequent_mask(len(src)).to(device)
            self.src_mask = mask

        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src, self.src_mask)
        # print("ENCODER OUTPUT")
        # print(len(output))
        output = output.mean(dim=1)  # aggregate across all tokens TODO: check if dim is correct
        # print("AGGREGATE")
        # print(len(output))
        output = self.classification_head(output) 
        # print("FINAL")
        # print(len(output))
        return output



class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term) #0::2 means starting with index 0, step = 2
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)